<img src="../Data/images/ZumiHeader.png" width=700>

# Applications of KNN: Red Light Green Light

<font size =3> Have you ever played the game "Red Light Green Light"? Here are the rules: a referee calls out "red light" or "green light" to players who are trying to race to a finish line. If the referee calls out red light, players must stop and freeze in their places. If the referee calls "green light", players race to the finish line. Whoever arrives first wins!
    
<img src="../Data/images/zumi_red_or_green.jpg" width=250>

In this lesson, you will teach Zumi that green means go and red means stop. This is pretty important for self-driving cars to know since cars stop (and go!) at traffic lights every day.</font>


### Step 1: Import Libraries
<font size =3> In this lesson, we are using the Zumi library for drive commands, the Camera library for training, the Screen library for printing text and emotions, and threading for running multiple processes at once. </font>


In [ ]:
from zumi.zumi import Zumi
from zumi.util.camera import Camera
from zumi.util.screen import Screen
from threading import Thread
from zumi.util.color_classifier import ColorClassifier

zumi = Zumi()
camera = Camera()
screen = Screen()

### Step 2: Gather data

<font size =3> Run the code below and train Zumi with green and red color cards. Name your demo ```red_green``` so you can find it again easily and use the following labels and key commands:

* Label: <font face="Courier">green</font>   
* Key Command: <font face="Courier">g</font>
* Label: <font face="Courier">red</font>
* Key Command: <font face="Courier">r</font> </font>


In [ ]:
camera.start_camera()
try:
    knn = ColorClassifier()
    train = knn.set_values()
    
    if train:
        print("Start gathering data. If you want to stop, press q")
        cnt = int(input("Type the amount of the pictures that you want to take at once : "))
        while True:
            knn.add_datas(camera, cnt)
            if knn.check_enough_datas(balance=True):
                break

        knn.save_data_set()
        knn.get_accuracy()
finally:
    camera.close()

### Step 3: Test the model
<font size =3> Before you start adding drive commands, test the model to check that your data has been trained correctly. Test it with some other objects too to see how reliable it is. If you need to, retrain the data under a different model name. </font> 

In [ ]:
try:
    print("Start predicting, press enter to predict!")
    camera.start_camera()
    knn.fit("hsv")
    while True:
        if input("press enter (or q to exit) : ") == 'q':
            break
        image = camera.capture()
        predict = knn.predict(image)
        print(predict)
finally:
    camera.close()


### Step 4: Program logic 
<font size =3> Once you’re finished testing your model, you can write out the logic with print statements.

When you call <font face="Courier">knn.predict(image)</font>, it returns its prediction with a label. For example, if you train the model with labels named <font face="Courier">blue</font> and <font face="Courier">yellow</font>, some code might look like this: <br> <br>

<font face="Courier">
while True:<br>
<span style="margin-left: 40px;">if input("Press "enter" to predict or "q" to exit.") == "q":</span> <br>
<span style="margin-left: 80px;">break</span> <br>
<span style="margin-left: 40px;">image = camera.capture()</span> <br>
<span style="margin-left: 40px;">predict = knn.predict(image)</span> <br>
<span style="margin-left: 40px;">if predict == "blue":</span> <br>
<span style="margin-left: 80px;">zumi.forward()</span> <br>
<span style="margin-left: 40px;">if predict == "yellow":</span> <br>
<span style="margin-left: 80px;">zumi.reverse()</span> <br>
camera.close() <br> <br>

</font>

The labels are in quotes because they are **string** data types. Strings are strings of characters, letters, or numbers that are treated as text. These labels must match your data labels EXACTLY. If you accidentally mistyped "blu" for your label but wrote your code to check if predict is equal to "blue", this code would never run.

Try writing the Red Light Green Light program with print statements. If the code says "CHANGE ME", make sure to make those changes to the correct labels for this project. </font> 


In [ ]:
try:
    print("Start predicting, press enter to predict!")
    camera.start_camera()
    knn.fit("hsv")
    
    while True:
        if input("press enter (or q to exit) : ") == 'q':
            break
        image = camera.capture()
        predict = knn.predict(image)
        if predict == "CHANGE ME":
            # Add a print statement here for driving forward
        if predict == "CHANGE ME":
            # Add a print statement here for stop

finally:
    camera.close()

### Step 5: Threading and go_straight()
<font size =3> Once you have tested your program logic with print statements, you need to add drive commands. Your first thought might be to write out your program logic like this: <br> <br>

<font face="Courier">
if predict == "green": <br>
    <span style="margin-left: 40px;">zumi.forward()</span> <br>
if predict == "red": <br>
    <span style="margin-left: 40px;">zumi.hard_brake() </span> <br>
</font>



<font face="Courier">Forward()</font> drives Zumi for a number of seconds that you specify and then stops, but you will want Zumi to stop driving *only* when she sees a red card. In this case, you need to use threading. This allows you to run multiple processes at the same time. Think of it like weaving two separate functions together so that they appear to be running simultaneously. <br> <br>

When you start a thread, it starts running in the background and lets the rest of the program continue running. You can use a **boolean**, or a true/false statement, to start or stop Zumi. The variable that you make for your boolean needs to be **global** so it can be used throughout the entire program instead of just one section. We named our variable <font face="Courier">is_green</font> and we set it to <font face="Courier">false</font> so that Zumi begins the program at a stop. We also defined our function <font face="Courier">continue_straight()</font>, which needs to be running in the background while <font face="Courier">is_green</font> is <font face="Courier">true</font>. Once Zumi sees red, <font face="Courier">is_green</font> becomes false, which will break the while loop (you can see that at “while is_green”) and cause Zumi to stop.

The cell below defines this function. Don't worry, Zumi won't start driving just yet. </font> 



In [ ]:
global is_green
is_green = False

def continue_straight():
    while is_green:
        zumi.go_straight(20, 0)
    zumi.stop(0)

<font size =3> Now you can weave these codes together with threading.  If Zumi sees green, you need to set the boolean to <font face="Courier">True</font> and begin the thread. If Zumi sees red, <font face="Courier">is_green</font> is now <font face="Courier">False</font>. <br> <br>

<font face="Courier">
if predict == "green": <br>
    <span style="margin-left: 40px;">is_green = True </span> <br>
<span style="margin-left: 40px;">drive_thread = Thread(target=continue_straight) </span> <br>
<span style="margin-left: 40px;">drive_thread.start() </span> <br>
if predict == "red": <br>
<span style="margin-left: 40px;">is_green = False </span> <br>
<span style="margin-left: 40px;">zumi.stop() </span> <br> <br>
</font>


Here is all of the code put together. Try it out! </font>
### Step 6: Final code

In [ ]:
try:
    print("Start predicting, press enter to predict!")
    camera.start_camera()
    knn.fit("hsv")
    
    while True:
        if input("press enter (or q to exit) : ") == 'q':
            break
        image = camera.capture()
        predict = knn.predict(image)
        if predict == "green":
            is_green = True
            drive_thread = Thread(target=continue_straight)
            drive_thread.start()
        if predict == "red":
            is_green = False
            zumi.stop()
finally:
    is_green = False
    zumi.stop()
    camera.close()

## Challenge
<font size =3> If you want, add more colors and have Zumi do different drive commands for each one! </font>